<a href="https://colab.research.google.com/github/yuhsuan0103/10901-algorithm/blob/master/1223NP%E5%95%8F%E9%A1%8C%E4%BB%8B%E7%B4%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

舉例：分團問題（clique problem）

問題類型：NP-complete

問題說明：團（clique）是一個圖中兩兩相鄰的一個頂點集或是一個完全子圖（complete subgraph）。分團問題是問一個圖中是否有大小是k以上的團。任意挑出k個點，可以簡單的判斷出這k個點是不是一個團[1]

解法：

1.暴力法。列舉圖中所有k個點的子集合，並檢查它是不是團。在一個有V個點的圖中用暴力法找大小是k的團至少要檢查V!/k!(V-k)!個子集合[1]

2動態規劃。採用動態規劃的思想就是將問題分段求解，可以將問題分為5段：求含有一個、兩個、三個、四個、五個元素的團，其中含有N元素的團必包含含有N-1個元素的團，故只需在上一段問題的答案的基礎上，嘗試給團新增新的元素。[2]


參考資料：

[1]分團問題 維基百科，自由的百科全書 https://zh.wikipedia.org/wiki/%E5%88%86%E5%9C%98%E5%95%8F%E9%A1%8C

[2]類動態規劃求解較小規模的最大團問題（Python實現）IT閱讀 https://www.itread01.com/content/1548192787.html

程式解如下，參考資料同[2]

In [3]:
V=[0,1,2,3,4]  
# E=[[1,3,4],[2,4],[4],[4],[]]  
E=[[1,3,4],[2,3,4],[4],[4],[]]  
  
  
  
###較小規模的最大團問題  
import copy  
  
def isConnected(u,v):  
    if u==-1 or v==-1:###虛擬節點-1與所有的節點都相連  
        return 1  
    edge_points=E[u]  
    if v in edge_points:  
        return 1  
    else:  
        edge_points = E[v]  
        if u in edge_points:  
            return 1  
        else:  
            return 0  
  
def isConnectedAll(clique,v):#判斷v是否和clique中所有節點相連  
    flag = 1  
    for i in clique:  
        if not isConnected(i,v):  
            flag =0  
            break  
    return flag  
  
class Step:  
    def __init__(self):  
        self.maxClique = [] #計算完畢時的解集（每個階段的實際結果）  
        self.cliqueList = []#計算時用的解集  
        self.maxnC = 0  
    def maxCliqn(self):#計算當前階段最大值  
        max = 0  
        for clique in self.cliqueList:  
            if max < len(clique):  
                max = len(clique)  
        return max  
    def isNew(self,clique): #判斷一個解組合是否已經存在於該階段的實際解集中  
        for cl in self.maxClique:#針對每個已存入的解集進行判斷  
            diff = list(set(clique).difference(set(cl)))  # 取解的差集  
            if (len(diff)):  
                continue      #差集不為空，說明不同，繼續迴圈  
            else:  
                return False  # 差集為空,說明有個解完全一樣，返回False 
  
        return True  
  
    def updateMaxClique(self):#更新當前階段的最大團數目  
        self.maxnC= self.maxCliqn()  
        for clique in self.cliqueList:  
            if(len(clique)==self.maxnC):  
                if self.isNew(clique):  
                    self.maxClique.append(clique)  
  
  
if __name__ == "__main__":  
    n = len(V)  
    solutions = {}  
    for i in range(0,n):  
        solutions[i]= Step()    #初始化n個階段  
    for v in V:  
       a = []  
       a.append(v)  
       solutions[0].cliqueList.append(a)  
    solutions[0].updateMaxClique()#設定初始值  
  
    for i in range(1,n):  
        #cliqList= solutions[i-1].maxClique  
        preData = solutions[i-1]  
        cliqList = preData.maxClique  
        preMax = preData.maxnC  
        for clique in cliqList:#針對前一階段的每個clique求解  
            for v in V:#針對所有的點  
                tempclique = copy.deepcopy(clique)##必須使用深拷貝  
                if not v in tempclique:#如果該clique沒有包含v  
                    if isConnectedAll(tempclique,v):#如果v與clique的所有點相連  
                        tempclique.append(v)#加入該點  
                        solutions[i].cliqueList.append(tempclique)#加入這個解  
        solutions[i].updateMaxClique()  
        if not len(solutions[i].maxClique):#如果已經找不到更多的點加入團，那麼後面的也不用計算了（比如找不到4個的團，那麼5個的團也沒必要再嘗試計算）  
            break  
  
    for i in range(0,n):  
        print("step"+str(i)+": "+str(solutions[i].maxClique))  
  
    for i in range(n-1,-1,-1):  
        solution = solutions[i]  
        if len(solution.maxClique):  
            maxn = solution.maxnC  
            print("最大團數目是"+ str(maxn)+"個")  
            print("最大團為:")  
            print(solution.maxClique)  
            break  

step0: [[0], [1], [2], [3], [4]]
step1: [[0, 1], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 4], [3, 4]]
step2: [[0, 1, 3], [0, 1, 4], [0, 3, 4], [1, 2, 4], [1, 3, 4]]
step3: [[0, 1, 3, 4]]
step4: []
最大團數目是4個
最大團為:
[[0, 1, 3, 4]]
